In [3]:
import cv2
import os
from keras import layers, Input
from keras.models import Model
from keras import models
from keras import optimizers
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import os
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
import numpy as np

In [21]:
total_data_final_2 = pd.read_csv('./total_data_final_2.csv')
total_data_final_2.columns
df = total_data_final_2[['folder_name','image_num','category_label','category_name',
                         'category_type','texture','shape','style']]


In [22]:
# 불필요 col 삭제
df = df.drop(['style','shape'], axis=1)

# category_type = 3 삭제
df = df[df.category_type != 3]

# NA 삭제
df = df.dropna(axis=0, how='any')
df.shape

# dropna 하면서 drop된 행의 index 비어버림 : index 초기화
df = df.reset_index(drop=True)
df


,folder_name,image_num,category_label,category_name,category_type,texture
0,Sheer_Pleated-Front_Blouse,img_00000009.jpg,3,Blouse,1,southwestern-patterned
1,Sheer_Pleated-Front_Blouse,img_00000035.jpg,3,Blouse,1,polka dot
2,Sheer_Pleated-Front_Blouse,img_00000036.jpg,3,Blouse,1,tribal
3,Sheer_Pleated-Front_Blouse,img_00000038.jpg,3,Blouse,1,chiffon floral
4,Sheer_Pleated-Front_Blouse,img_00000040.jpg,3,Blouse,1,zig
...,...,...,...,...,...,...
64746,Destroyed_Denim_Cutoffs,img_00000015.jpg,24,Cutoffs,2,diamond
64747,Destroyed_Denim_Cutoffs,img_00000016.jpg,24,Cutoffs,2,diamond print
64748,Destroyed_Denim_Cutoffs,img_00000017.jpg,24,Cutoffs,2,diamond print
64749,Destroyed_Denim_Cutoffs,img_00000018.jpg,24,Cutoffs,2,paint


In [23]:
# 폴더 생성 함수

def createfolder(foler_dir):
    try:
        if not os.path.exists(folder_dir):
            os.makedirs(folder_dir)
    except OSError:
        print('Error Creating directory'+folder_dir)


In [34]:
# texture_name 폴더 만들기

texture_list = list(df['texture'].unique())
print(f'label, class 갯수: {len(texture_list)}')

for folder_name in texture_list:
    a = 'C:/Users/student/Desktop/texture/'
    folder_dir = a + folder_name
    createfolder(folder_dir)

label, class 갯수: 145


In [37]:
# category_name에 맞춰서 사진 다시 저장

for _ in range(df.shape[0]):
    num = str(_)
    texture_name = df['texture'][_]
    img_folder = df['folder_name'][_] 
    img_name = df['image_num'][_] 
    full_dir = str('D:/Project/img/' + img_folder + '/' + img_name) 
    
    img = cv2.imread(full_dir)
    img = cv2.resize(img, (150,150))
    
    new_dir = str('C:/Users/student/Desktop/texture/'+ texture_name)
    new_name = new_dir + '/' + texture_name + '_' + num + '.jpg'
    cv2.imwrite(new_name, img) 
    
    if _ % 5000 == 0:
        print(_)


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000


## texture로 학습

In [84]:
# generator = tf.keras.preprocessing.image.ImageDataGenerator(
#     rotation_range = 20,
#     width_shift_range = 0.2,
#     height_shift_range = 0.2,
#     rescale = 1. / 255)

In [38]:
model = Sequential()
model.add(Conv2D(16, (3, 3), padding='same', use_bias=False, input_shape=(150,150, 3)))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(32, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dropout(0.2))
 
model.add(Dense(512, activation='relu'))
model.add(Dense(145, activation='softmax'))   #label, calss 갯수
model.summary()
 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 150, 150, 16)      432       
_________________________________________________________________
batch_normalization_9 (Batch (None, 150, 150, 16)      48        
_________________________________________________________________
activation_9 (Activation)    (None, 150, 150, 16)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 38, 38, 16)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 38, 38, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 38, 38, 32)        4608      
_________________________________________________________________
batch_normalization_10 (Batc (None, 38, 38, 32)        96        
__________

In [39]:
# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


# 데이터 셋
train_datagen = ImageDataGenerator(rescale=1./255,
#                                    rotation_range=10,
#                                    width_shift_range=0.2,
#                                    height_shift_range=0.2,
#                                    shear_range=0.7,
#                                    zoom_range=[0.9, 2.2],
#                                    horizontal_flip=True,
#                                    vertical_flip=True,
#                                    fill_mode='nearest',
                                   validation_split=0.33)
 
training_set = train_datagen.flow_from_directory('C:/Users/student/Desktop/texture',
                                                 target_size = (150,150),
                                                 batch_size = 15,
                                                 class_mode = 'categorical',
                                                 subset="training"
                                                )
validation_set = train_datagen.flow_from_directory('C:/Users/student/Desktop/texture',
                                                 target_size = (150,150),
                                                 batch_size = 10,
                                                 class_mode = 'categorical',
                                                 subset="validation"
                                                  )
 

Found 43451 images belonging to 145 classes.
Found 21300 images belonging to 145 classes.


In [41]:
from keras.callbacks import CSVLogger
 
csv_logger = CSVLogger('./log.csv', append=True, separator=';')
 
 
hist = model.fit_generator(training_set,
                         steps_per_epoch = 20, #20
                         epochs = 1000,  #1000
                         validation_data = validation_set,
                         validation_steps = 10,
                         callbacks=[csv_logger])
 
 

Epoch 1/1000
20/20 [==============================] - 13s 670ms/step - loss: 4.6094 - acc: 0.0467 - val_loss: 4.4905 - val_acc: 0.0600
Epoch 2/1000
20/20 [==============================] - 13s 663ms/step - loss: 4.6332 - acc: 0.0533 - val_loss: 4.4561 - val_acc: 0.0600
Epoch 3/1000
20/20 [==============================] - 13s 663ms/step - loss: 4.5214 - acc: 0.0633 - val_loss: 4.4297 - val_acc: 0.0600
Epoch 4/1000
20/20 [==============================] - 14s 695ms/step - loss: 4.6358 - acc: 0.0433 - val_loss: 4.4560 - val_acc: 0.0800
Epoch 5/1000
20/20 [==============================] - 13s 668ms/step - loss: 4.5602 - acc: 0.0667 - val_loss: 4.4599 - val_acc: 0.0600
Epoch 6/1000
20/20 [==============================] - 13s 669ms/step - loss: 4.5153 - acc: 0.0767 - val_loss: 4.4246 - val_acc: 0.0600
Epoch 7/1000
20/20 [==============================] - 13s 669ms/step - loss: 4.5502 - acc: 0.0533 - val_loss: 4.4598 - val_acc: 0.0300
Epoch 8/1000
20/20 [==============================] - 1

20/20 [==============================] - 13s 664ms/step - loss: 4.4456 - acc: 0.0633 - val_loss: 4.4291 - val_acc: 0.0700
Epoch 62/1000
20/20 [==============================] - 13s 662ms/step - loss: 4.4050 - acc: 0.0767 - val_loss: 4.4413 - val_acc: 0.0600
Epoch 63/1000
20/20 [==============================] - 13s 665ms/step - loss: 4.5185 - acc: 0.0600 - val_loss: 4.4739 - val_acc: 0.1000
Epoch 64/1000
20/20 [==============================] - 13s 666ms/step - loss: 4.5524 - acc: 0.0333 - val_loss: 4.4376 - val_acc: 0.0900
Epoch 65/1000
20/20 [==============================] - 13s 657ms/step - loss: 4.4934 - acc: 0.0533 - val_loss: 4.4863 - val_acc: 0.0300
Epoch 66/1000
20/20 [==============================] - 13s 668ms/step - loss: 4.4679 - acc: 0.0933 - val_loss: 4.4690 - val_acc: 0.0600
Epoch 67/1000
20/20 [==============================] - 14s 684ms/step - loss: 4.3931 - acc: 0.1000 - val_loss: 4.4442 - val_acc: 0.0900
Epoch 68/1000
20/20 [==============================] - 13s 671

20/20 [==============================] - 14s 686ms/step - loss: 4.4237 - acc: 0.1033 - val_loss: 4.3863 - val_acc: 0.0800
Epoch 122/1000
20/20 [==============================] - 13s 673ms/step - loss: 4.3516 - acc: 0.0833 - val_loss: 4.4162 - val_acc: 0.0800
Epoch 123/1000
20/20 [==============================] - 14s 693ms/step - loss: 4.4012 - acc: 0.0800 - val_loss: 4.3840 - val_acc: 0.1100
Epoch 124/1000
20/20 [==============================] - 14s 684ms/step - loss: 4.3586 - acc: 0.0967 - val_loss: 4.3908 - val_acc: 0.0900
Epoch 125/1000
20/20 [==============================] - 14s 686ms/step - loss: 4.4405 - acc: 0.0633 - val_loss: 4.4351 - val_acc: 0.0900
Epoch 126/1000
20/20 [==============================] - 14s 689ms/step - loss: 4.4053 - acc: 0.0833 - val_loss: 4.4242 - val_acc: 0.1000
Epoch 127/1000
20/20 [==============================] - 13s 673ms/step - loss: 4.4350 - acc: 0.0600 - val_loss: 4.4301 - val_acc: 0.0800
Epoch 128/1000
20/20 [==============================] - 

Epoch 181/1000


KeyboardInterrupt: 

In [ ]:
from keras.models import load_model
 
model.save('cnn_attraction_keras_model.h5')
 
# output = classifier.predict_generator(test_set, steps=5)
# print(test_set.class_indices)
# print(output)
 
# 모델 평가하기
print("-- Evaluate --")
 
scores = model.evaluate_generator(
            validation_set,
            steps = 10)
 
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
 
# 모델 예측하기
print("-- Predict --")
 
output = model.predict_generator(
            validation_set,
            steps = 100)
print(validation_set.class_indices)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
 

In [ ]:
import matplotlib.pyplot as plt
 
fig, loss_ax = plt.subplots()
 
acc_ax = loss_ax.twinx()
 
loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
#loss_ax.set_ylim([0.0, 0.5])
 
acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
#acc_ax.set_ylim([0.8, 1.0])
 
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')
 
loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')
 
plt.show()